In [1]:
%load_ext autoreload
%autoreload 2

# Set up

In [2]:
# import necessary packages 
import os 
import numpy as np 
from pathlib import Path
import itertools
import pandas as pd

### Set up parameters

In [3]:
# Define parameter grid for MLP
param_grid = {
    'hidden_sizes': [[64, 32], [128, 64], [32, 16]],
    'learning_rate': [0.001, 0.01, 0.1],
    'activation': ['relu', 'sigmoid']
}

# Generate all combinations of parameters
param_combinations = list(itertools.product(
    param_grid['hidden_sizes'],
    param_grid['learning_rate'],
    param_grid['activation']
))

### Set up data

In [4]:
# Get data directories
data_folder = '../data'
train_dir = os.path.join(data_folder, 'train.csv')
test_dir = os.path.join(data_folder, 'test.csv')
val_dir = os.path.join(data_folder, 'val.csv')

In [5]:
# Load data
def load_data(data_dir): 
    df = pd.read_csv(data_dir)
    X = df.drop(columns=['TARGET']) 
    y = df['TARGET'] 
    return X, y

X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)
X_val, y_val = load_data(val_dir)

### Set up folder for logging

In [6]:
experiment_dir = Path('./output/MLP')
experiment_dir.mkdir(parents=True, exist_ok=True)

# MLP from scratch

In [7]:
# Set up system path
import sys
project_root = Path('.').resolve().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

import importlib
import train.mlp_trainer
import model.mlp
importlib.reload(model.mlp)
importlib.reload(train.mlp_trainer)
from model.mlp import MLP
from train.mlp_trainer import MLPTrainer

In [8]:
# Placeholder for results
results = []

In [9]:
for i, (hidden_sizes, learning_rate, activation) in enumerate(param_combinations):
    params = {
        'hidden_sizes': hidden_sizes,
        'learning_rate': learning_rate,
        'activation': activation,
        'epochs': 1000,
        'threshold': 0.5,
        'output_size': 1
    }

    config = {'params': params}

    # Train
    trainer = MLPTrainer(config)
    eval_metrics = trainer.train(X_train, y_train, X_val, y_val, experiment_dir)

    # Record results
    results.append({
        'hidden_sizes': hidden_sizes,
        'learning_rate': learning_rate,
        'activation': activation,
        'accuracy': eval_metrics['accuracy'],
        'precision': eval_metrics['precision'],
        'recall': eval_metrics['recall'],
        'f1': eval_metrics['f1']
    })

# Convert to DataFrame
scratch_results_df = pd.DataFrame(results)

# Save to CSV
scratch_results_df.to_csv(experiment_dir / 'scratch_finetune_results.csv', index=False)

# (Optional) Print top configurations based on F1-score
print(scratch_results_df.sort_values(by='f1', ascending=False).head())

2025-06-03 16:45:45,871 - INFO - Training MLP with parameters: {'hidden_sizes': [64, 32], 'learning_rate': 0.001, 'activation': 'relu', 'epochs': 1000, 'threshold': 0.5, 'output_size': 1}
2025-06-03 16:45:45,903 - INFO - Input data shape: (132669, 16)


Epoch 100/1000, Train Loss: 0.7610, Train Acc: 0.4161, Val Loss: 0.7615, Val Acc: 0.4163


KeyboardInterrupt: 

In [ ]:
# Get best hyperparameter set
best_result = scratch_results_df.loc[scratch_results_df['f1'].idxmax()]
best_params = {
    'hidden_sizes': best_result['hidden_sizes'],
    'learning_rate': best_result['learning_rate'],
    'activation': best_result['activation']
}

print("Best parameters based on F1-score:")
print(best_params)
print("\nValidation metrics:")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Precision: {best_result['precision']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"F1-score: {best_result['f1']:.4f}")

In [ ]:
# Plot into chart

# MLP Sklearn

In [ ]:
# Import libraries
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import logging

In [ ]:
results = []

for i, (hidden_sizes, learning_rate, activation) in enumerate(param_combinations):
    params = {
        'hidden_layer_sizes': tuple(hidden_sizes),
        'learning_rate_init': learning_rate,
        'activation': activation,
        'max_iter': 1000,
        'random_state': 42
    }

    # Log model configuration
    logging.info(f"Training MLP using sklearn with parameters: {params}")

    # Initialize sklearn MLP model
    model = MLPClassifier(**params)

    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    # Calculate metrics
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_pred_train),
        'precision': precision_score(y_train, y_pred_train, average='macro'),
        'recall': recall_score(y_train, y_pred_train, average='macro'),
        'f1': f1_score(y_train, y_pred_train, average='macro')
    }

    val_metrics = {
        'accuracy': accuracy_score(y_val, y_pred_val),
        'precision': precision_score(y_val, y_pred_val, average='macro'),
        'recall': recall_score(y_val, y_pred_val, average='macro'),
        'f1': f1_score(y_val, y_pred_val, average='macro')
    }

    # Record results
    results.append({
        'hidden_sizes': hidden_sizes,
        'learning_rate': learning_rate,
        'activation': activation,
        'accuracy': val_metrics['accuracy'],
        'precision': val_metrics['precision'],
        'recall': val_metrics['recall'],
        'f1': val_metrics['f1']
    })

    # Log training metrics
    logging.info("Training metrics:")
    logging.info(f"Accuracy: {train_metrics['accuracy']:.4f}")
    logging.info(f"Precision: {train_metrics['precision']:.4f}")
    logging.info(f"Recall: {train_metrics['recall']:.4f}")
    logging.info(f"F1 Score: {train_metrics['f1']:.4f}")
    logging.info("Classification Report:\n" + classification_report(y_train, y_pred_train))

    # Log validation metrics
    logging.info("Validation results:")
    logging.info(f"Accuracy: {val_metrics['accuracy']:.4f}")
    logging.info(f"Precision: {val_metrics['precision']:.4f}")
    logging.info(f"Recall: {val_metrics['recall']:.4f}")
    logging.info(f"F1 Score: {val_metrics['f1']:.4f}")
    logging.info("Classification Report:\n" + classification_report(y_val, y_pred_val))

# Convert to DataFrame
sklearn_results_df = pd.DataFrame(results)

# Save to CSV
sklearn_results_df.to_csv(experiment_dir / 'sklearn_finetune_results.csv', index=False)

# Print top 5 based on F1-score
print(sklearn_results_df.sort_values(by='f1', ascending=False).head())

In [ ]:
# Get best hyperparameter set
best_result = sklearn_results_df.loc[sklearn_results_df['f1'].idxmax()]
best_params = {
    'hidden_sizes': best_result['hidden_sizes'],
    'learning_rate': best_result['learning_rate'],
    'activation': best_result['activation']
}

print("Best parameters based on F1-score:")
print(best_params)
print("\nValidation metrics:")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Precision: {best_result['precision']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"F1-score: {best_result['f1']:.4f}")

In [ ]:
# Plot into charts using finetune_results.csv